In [152]:
import pandas as pd
df = pd.read_csv('Peru_2019_AudioMoth_Data_Full.csv', low_memory=False)

In [153]:
#only get rows where the duration is atleast 60
df=df[df['Duration']>=60]

In [154]:
#get the original file size to compare it with the new dataframe's filesize later on
originalDfSize=df['FileSize'].sum()/1000000000
print("File size in original df is", originalDfSize, "GB")

File size in original df is 4317.265904676 GB


In [155]:
#remove all moth codes where there are less than 24 occurences in the df
df = df.groupby('AudioMothCode').filter(lambda x: len(x) > 24)

In [156]:
#convert to datetime format
times = pd.to_datetime(df['StartDateTime'], dayfirst=True)
# get only hour information and save it as a new column in df
df['Hour'] = times.dt.hour

In [157]:
#create a new df where the column information is the same as the previous data frame
stratified_sample_df = pd.DataFrame(columns=df.columns)

In [158]:
#get a list of all the unique moth codes to loop through later
MothCode_list = df['AudioMothCode'].unique().tolist()
#loop through all the different moth codes
for i in MothCode_list:
    #created a new table of just a specific mothcode. This way, when the query is run in the nested loop, it only goes through this subset to improve runtime 
    mothcodedf=df[(df['AudioMothCode'] == i)]
    #Loop from 0 to 23 for each moth code to include every hour
    for j in range(24):
        #retrieve all the rows from that specific mothcode whose hour is the same as the intended hour
        currdf=mothcodedf[(mothcodedf['Hour'] == j)]
        #if there is atleast 1 row in the current hour dataframe, get a random sample of 1 from the current dataframe
        if (currdf.shape[0]>0):
            sample = currdf.sample(n = 1)
            #append the row to the end of the new df
            stratified_sample_df = pd.concat([stratified_sample_df, sample], ignore_index=True)
        #if there is no hour information (ie for all rows with mothcode WWF-#), then grab a random sample from the larger mothcodes df
        else:
            sample = mothcodedf.sample(n = 1)
            stratified_sample_df = pd.concat([stratified_sample_df, sample], ignore_index=True)

In [159]:
#verify that the new stratified sample is much smaller than the original
#because none of the WWF moth codes have time-date information, they have not been added to csv, so the filesize for the newdf is not 34.3
newDfSize=stratified_sample_df['FileSize'].sum()/1000000000
print("File size in new df is", newDfSize, "GB")
print("Difference in file size between original and new is", originalDfSize-newDfSize, "GB")

File size in new df is 34.28376768 GB
Difference in file size between original and new is 4282.982136996 GB


In [160]:
#converts to a csv called output.csv
stratified_sample_df.to_csv('output.csv', index=False)